## OTF Workouts


### Imports

In [1]:
from pmaw import PushshiftAPI
import pandas as pd
import datetime as dt
import csv
import re 
from pprint import pprint
api = PushshiftAPI()
import os, glob
import secrets
import nltk
import numpy as np
from string import punctuation
from collections import Counter
from nltk.corpus import stopwords
sw = stopwords.words('english')

## Pull Comments

In [5]:
before = int(dt.datetime(2021,11,30,0,0).timestamp()) 
after = int(dt.datetime(2012,1,1,0,0).timestamp())

I wanted to pull information from the r/Orangetheory subreddit that contained information on the daily work out. The sub has gone through several variations of how this information is posted/shared. Currently, there is a automated post with the date and a generic post, once the mods have the workout intel for the day they post the information as a post, by splat_bot by the user u/splat_bot (which is a bot). 

I initially tried using PRAW API but was not able to get all the data I wanted due to limitations of the API.

In [6]:
# Generate comments from splat_bot
subreddit="orangetheory"
username =  'splat_bot' #otfmonitorbot
limit=1000000
sb_comments = api.search_comments(subreddit=subreddit, author=username, limit=limit, before=before, after=after)
print(f'Retrieved {len(sb_comments)} comments from OTF Sub by Splat_Bot')

INFO:pmaw.PushshiftAPIBase:993049 result(s) not found in Pushshift
INFO:pmaw.PushshiftAPIBase:Checkpoint:: Success Rate: 100.00% - Requests: 94 - Batches: 10 - Items Remaining: 0
INFO:pmaw.PushshiftAPIBase:Total:: Success Rate: 100.00% - Requests: 94 - Batches: 10 - Items Remaining: 0
Retrieved 6951 comments from OTF Sub by Splat_Bot


In [7]:
# Generate comments from otfmonitorbot
subreddit="orangetheory"
username =  'otfmonitorbot'
limit=1000000
om_comments = api.search_comments(subreddit=subreddit, author=username, limit=limit, before=before, after=after)
print(f'Retrieved {len(sb_comments)} comments from OTF Sub by otfmonitorbot')

INFO:pmaw.PushshiftAPIBase:999985 result(s) not found in Pushshift
INFO:pmaw.PushshiftAPIBase:Total:: Success Rate: 100.00% - Requests: 8 - Batches: 1 - Items Remaining: 0
Retrieved 6951 comments from OTF Sub by otfmonitorbot


In [11]:
#total comments pulled
#len(om_comments) + len(sb_comments)

Create Pandas DF of comments from r/splat_bot

In [12]:
sb_df = pd.DataFrame(sb_comments)

# preview the comments data
sb_df 

,all_awardings,associated_award,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,...,collapsed_reason_code,controversiality,gilded,retrieved_utc,score_hidden,subreddit_name_prefixed,subreddit_type,unrepliable_reason,updated_utc,author_cakeday
0,[],NaN,splat_bot,#dadada,None,"[{'a': ':splat:', 'e': 'emoji', 'u': 'https://...",50e2e98e-e160-11e8-acdf-0e90e4c5f6ac,:splat: Mod | AI :robot:,dark,richtext,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],NaN,splat_bot,#dadada,None,"[{'a': ':splat:', 'e': 'emoji', 'u': 'https://...",50e2e98e-e160-11e8-acdf-0e90e4c5f6ac,:splat: Mod | AI :robot:,dark,richtext,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],NaN,splat_bot,#dadada,None,"[{'a': ':splat:', 'e': 'emoji', 'u': 'https://...",50e2e98e-e160-11e8-acdf-0e90e4c5f6ac,:splat: Mod | AI :robot:,dark,richtext,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],NaN,splat_bot,#dadada,None,"[{'a': ':splat:', 'e': 'emoji', 'u': 'https://...",50e2e98e-e160-11e8-acdf-0e90e4c5f6ac,:splat: Mod | AI :robot:,dark,richtext,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],NaN,splat_bot,#dadada,None,"[{'a': ':splat:', 'e': 'emoji', 'u': 'https://...",50e2e98e-e160-11e8-acdf-0e90e4c5f6ac,:splat: Mod | AI :robot:,dark,richtext,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6946,[],NaN,splat_bot,#dadada,None,"[{'a': ':partyparrot:', 'e': 'emoji', 'u': 'ht...",50e2e98e-e160-11e8-acdf-0e90e4c5f6ac,:partyparrot:Mod | AI,dark,richtext,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6947,[],NaN,splat_bot,#dadada,None,"[{'a': ':partyparrot:', 'e': 'emoji', 'u': 'ht...",50e2e98e-e160-11e8-acdf-0e90e4c5f6ac,:partyparrot:Mod | AI,dark,richtext,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6948,[],NaN,splat_bot,#dadada,None,"[{'a': ':partyparrot:', 'e': 'emoji', 'u': 'ht...",50e2e98e-e160-11e8-acdf-0e90e4c5f6ac,:partyparrot:Mod | AI,dark,richtext,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6949,[],NaN,splat_bot,#dadada,None,"[{'a': ':partyparrot:', 'e': 'emoji', 'u': 'ht...",50e2e98e-e160-11e8-acdf-0e90e4c5f6ac,:partyparrot:Mod | AI,dark,richtext,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Create Pandas DF of comments from r/otfmonitorbot

In [13]:
om_df = pd.DataFrame(om_comments)

# preview the comments data
om_df 

,all_awardings,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,author_fullname,...,retrieved_on,score,send_replies,stickied,subreddit,subreddit_id,total_awards_received,updated_utc,distinguished,gilded
0,[],otfmonitorbot,None,None,[],None,None,None,text,t2_3qir1v74,...,1559884364,1,True,False,orangetheory,t5_39w4v,0,1560184959,NaN,NaN
1,[],otfmonitorbot,None,None,[],None,None,None,text,t2_3qir1v74,...,1559797894,1,False,True,orangetheory,t5_39w4v,0,1560153352,moderator,NaN
2,[],otfmonitorbot,None,None,[],None,None,None,text,t2_3qir1v74,...,1559797891,60,False,False,orangetheory,t5_39w4v,0,1560153350,moderator,1.0
3,[],otfmonitorbot,None,None,[],None,None,None,text,t2_3qir1v74,...,1559797887,1,False,True,orangetheory,t5_39w4v,0,1560153348,moderator,NaN
4,[],otfmonitorbot,None,None,[],None,None,None,text,t2_3qir1v74,...,1559797883,33,False,False,orangetheory,t5_39w4v,0,1560153348,moderator,NaN
5,[],otfmonitorbot,None,None,[],None,None,None,text,t2_3qir1v74,...,1559711491,1,True,False,orangetheory,t5_39w4v,0,1560121379,NaN,NaN
6,[],otfmonitorbot,None,None,[],None,None,None,text,t2_3qir1v74,...,1559625052,1,True,False,orangetheory,t5_39w4v,0,1560093400,NaN,NaN
7,[],otfmonitorbot,None,None,[],None,None,None,text,t2_3qir1v74,...,1559625048,74,True,False,orangetheory,t5_39w4v,0,1560093400,NaN,NaN
8,[],otfmonitorbot,None,None,[],None,None,None,text,t2_3qir1v74,...,1559538853,74,True,False,orangetheory,t5_39w4v,0,1560070423,NaN,NaN
9,[],otfmonitorbot,None,None,[],None,None,None,text,t2_3qir1v74,...,1559452290,45,True,False,orangetheory,t5_39w4v,0,1560044811,NaN,NaN


Make DF of body and date

In [14]:
# Pull just columns wanted
workout_df = pd.DataFrame(sb_df, columns = ['body', 'created_utc'])
workout_df

,body,created_utc
0,This submission was removed by the moderators....,1575673836
1,This submission was removed by the moderators....,1575671564
2,This submission was removed by the moderators....,1575667868
3,This submission was removed by the moderators....,1575632230
4,Reposting content courtesy of /u/thmpsne. \n[...,1575612064
...,...,...
6946,I found a possible match to a frequently-asked...,1568223541
6947,This submission was removed by the moderators....,1568216264
6948,This submission was removed by the moderators....,1568215615
6949,I found a possible match to a frequently-asked...,1568209872


In [15]:
# Pull just columns wanted
workout_df2 = pd.DataFrame(om_df, columns = ['body', 'created_utc'])
workout_df2

,body,created_utc
0,This thread is now locked. Please continue the...,1559884362
1,This thread is now locked. Please continue the...,1559797884
2,Reposting early intel courtesy of /u/LivingMem...,1559797880
3,This thread is now locked. Please continue the...,1559797876
4,Reposting early intel courtesy of /u/LivingMem...,1559797873
5,This thread is now locked. Please continue the...,1559711489
6,This thread is now locked. Please continue the...,1559625051
7,Reposting early intel courtesy of /u/matthotli...,1559625047
8,Reposting early intel courtesy of /u/LivingMem...,1559538852
9,Reposting early intel courtesy of /u/LivingMem...,1559452289


Save DFs as CSV's

In [16]:
workout_df.to_csv('otf_splatbot.csv', header=True, index=False, columns=list(workout_df.axes[1]))

In [17]:
workout_df2.to_csv('otf_otfmonitor.csv', header=True, index=False, columns=list(workout_df2.axes[1]))

Combine the two CSVs

In [18]:
path = "\\Users\heath\\OneDrive - The University of Montana\\Applied_Data_Analytics\\data_project_2\\original_pulls\\"

all_files = glob.glob(os.path.join(path, "otf*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv( "combined_wos.csv", index = None)

Read in the combined file

In [19]:
#workouts = open("combined_wos.csv",'r', encoding='utf-8').read()
workout = pd.read_csv("combined_wos.csv") 
workout["created_utc"] = pd.to_datetime(workout["created_utc"], unit= 's')
workout['date'] = [d.date() for d in workout['created_utc']]
workout.drop('created_utc', axis=1, inplace=True)
workout['date'] = pd.to_datetime(workout.date)
workout

,body,date
0,This thread is now locked. Please continue the...,2019-06-07
1,This thread is now locked. Please continue the...,2019-06-06
2,Reposting early intel courtesy of /u/LivingMem...,2019-06-06
3,This thread is now locked. Please continue the...,2019-06-06
4,Reposting early intel courtesy of /u/LivingMem...,2019-06-06
...,...,...
6961,I found a possible match to a frequently-asked...,2019-09-11
6962,This submission was removed by the moderators....,2019-09-11
6963,This submission was removed by the moderators....,2019-09-11
6964,I found a possible match to a frequently-asked...,2019-09-11


### Clean up

In [20]:
#lowercase
workout.body = workout.body.str.lower()

# Remove non alpha chars
workout.body = workout.body.apply(lambda x:' '.join(re.findall(r'\w+', x)))

Clean comments that are not related to workouts

In [21]:
# Drop rows that don't contain 'reposting' which are comments that contain workouts
clean_workout = workout[workout['body'].apply(lambda x: 'reposting' in x)]

In [22]:
clean_workout

,body,date
2,reposting early intel courtesy of u livingmemo...,2019-06-06
4,reposting early intel courtesy of u livingmemo...,2019-06-06
7,reposting early intel courtesy of u matthotlip...,2019-06-04
8,reposting early intel courtesy of u livingmemo...,2019-06-03
9,reposting early intel courtesy of u livingmemo...,2019-06-02
...,...,...
6883,reposting content courtesy of u kaorian saturd...,2019-10-19
6899,reposting content courtesy of u superspykay fr...,2019-10-18
6900,reposting content courtesy of u livingmemory 3...,2019-10-18
6912,reposting content courtesy of u livingmemory 3...,2019-10-17


In [ ]:
#This works but found more efficent way after doing this

#workout_df = workout_df[workout_df["body"].str.contains("i found some information")==False]
#workout_df = workout_df[workout_df["body"].str.contains("this submission")==False]
#workout_df = workout_df[workout_df["body"].str.contains("this thread is now locked")==False]
#workout_df = workout_df[workout_df["body"].str.contains("please see previous message")==False]
#workout_df = workout_df[workout_df["body"].str.contains("i found a possible match")==False]
#workout_df = workout_df[workout_df["body"].str.contains("this is a very common question")==False]
#workout_df = workout_df[workout_df["body"].str.contains("it looks like")==False]workout_df = workout_df[workout_df["body"].str.contains("reminder from the mods")==False]
#workout_df = workout_df[workout_df["body"].str.contains("my heart beats")==False]
#workout_df = workout_df[workout_df["body"].str.contains("note from the moderators")==False]
#workout_df = workout_df[workout_df["body"].str.contains("oh no not the delete")==False]
#workout_df = workout_df[workout_df["body"].str.contains("per the wiki")==False]

Save DFs as CSVs

In [49]:
# Save cleaned dataframe as a csv
clean_workout.to_csv('clean_workouts.csv', header=True, index=False, columns=list(clean_workouts.axes[1]))

In [23]:
len(clean_workout)

907

In [50]:
#bring in my stats
#hp_stats = open("updated_otfstats.csv",'r', encoding='utf-8').read()
hp_stats = pd.read_csv("updated_otfstats.csv") 
hp_stats['Date'] = pd.to_datetime(hp_stats.Date)
hp_stats

,Class Type,Focus,Coach,Time,Date,Day of Week,Grey,Blue,Green,Orange,...,Splat Points,Avg HR,Max HR,Total Calories,Distance,Avg Speed,Max Speed,Avg Pace,Elevation,Notes
0,2G,ESP,Brittney,11:15 AM,2017-10-28,Saturday,1,1,7,36,...,43,169,185.0,586,NaN,NaN,NaN,NaN,NaN,NaN
1,2G,ESP,Shania,11:15 AM,2017-11-04,Saturday,2,1,21,33,...,34,162,180.0,615,NaN,NaN,NaN,NaN,NaN,NaN
2,3G,Endurance,Other,5:00 AM,2017-11-20,Monday,0,3,19,38,...,41,165,184.0,695,NaN,NaN,NaN,NaN,NaN,NaN
3,2G,Power,Gabriella,5:30 PM,2017-11-22,Wednesday,1,4,36,16,...,16,155,177.0,572,NaN,NaN,NaN,NaN,NaN,NaN
4,2G,Power,Bri,7:00 AM,2017-11-25,Saturday,1,13,36,10,...,10,150,173.0,560,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2G,ESP,Brittney,6:45 PM,2021-07-12,Monday,1,10,26,16,...,19,143,178.0,500,1.75,3.7,6.4,16:00:00,405.4,NaN
236,2G,Power,Shania,6:45 PM,2021-07-15,Thursday,2,4,25,24,...,26,147,175.0,522,1.76,3.7,5,16:04:00,408.3,"The ""Chipper"""
237,3G,Power,Katie,8:15 AM,2021-07-17,Saturday,4,6,24,23,...,24,144,174.0,508,1.37,4.2,7,14:14:00,71.3,NaN
238,2G,Power,Shania,6:45 PM,2021-07-22,Thursday,1,6,25,15,...,24,148,185.0,524,1.77,4.4,6.4,13:40:00,99.6,12 Minute Prep


In [ ]:
#hp_stats.rename(columns={'Date': 'date'}, inplace=True)

In [42]:
hp_stats

,Class Type,Focus,Coach,Time,Date,Day of Week,Grey,Blue,Green,Orange,...,Splat Points,Avg HR,Max HR,Total Calories,Distance,Avg Speed,Max Speed,Avg Pace,Elevation,Notes
0,2G,ESP,Brittney,11:15 AM,2017-10-28,Saturday,1,1,7,36,...,43,169,185.0,586,NaN,NaN,NaN,NaN,NaN,NaN
1,2G,ESP,Shania,11:15 AM,2017-11-04,Saturday,2,1,21,33,...,34,162,180.0,615,NaN,NaN,NaN,NaN,NaN,NaN
2,3G,Endurance,Other,5:00 AM,2017-11-20,Monday,0,3,19,38,...,41,165,184.0,695,NaN,NaN,NaN,NaN,NaN,NaN
3,2G,Power,Gabriella,5:30 PM,2017-11-22,Wednesday,1,4,36,16,...,16,155,177.0,572,NaN,NaN,NaN,NaN,NaN,NaN
4,2G,Power,Bri,7:00 AM,2017-11-25,Saturday,1,13,36,10,...,10,150,173.0,560,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2G,ESP,Brittney,6:45 PM,2021-07-12,Monday,1,10,26,16,...,19,143,178.0,500,1.75,3.7,6.4,16:00:00,405.4,NaN
236,2G,Power,Shania,6:45 PM,2021-07-15,Thursday,2,4,25,24,...,26,147,175.0,522,1.76,3.7,5,16:04:00,408.3,"The ""Chipper"""
237,3G,Power,Katie,8:15 AM,2021-07-17,Saturday,4,6,24,23,...,24,144,174.0,508,1.37,4.2,7,14:14:00,71.3,NaN
238,2G,Power,Shania,6:45 PM,2021-07-22,Thursday,1,6,25,15,...,24,148,185.0,524,1.77,4.4,6.4,13:40:00,99.6,12 Minute Prep


### Join my stats with the workout information
Below I combined data I have collected from my OTF workouts such as average heart rate, splat points,and distance. In the summer class I worked with that data, after the web scraping assignments I wanted see if I could include the details about the work out too, below I was able to combine the comments I collected and join them on the dates I have data for.

There are missing variables in my stats data as the information they collect/share has changed over time.

In [62]:
stats_w_wo = pd.merge(left=hp_stats, right=clean_workout, how='left', left_on='Date', right_on='date')
stats_w_wo.rename(columns={'body': 'Workout Details'}, inplace=True)


In [56]:
stats_w_wo.drop('date', axis=1, inplace=True)

In [57]:
stats_w_wo

,Class Type,Focus,Coach,Time,Date,Day of Week,Grey,Blue,Green,Orange,...,Avg HR,Max HR,Total Calories,Distance,Avg Speed,Max Speed,Avg Pace,Elevation,Notes,Workout Details
0,2G,ESP,Brittney,11:15 AM,2017-10-28,Saturday,1,1,7,36,...,169,185.0,586,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2G,ESP,Shania,11:15 AM,2017-11-04,Saturday,2,1,21,33,...,162,180.0,615,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3G,Endurance,Other,5:00 AM,2017-11-20,Monday,0,3,19,38,...,165,184.0,695,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2G,Power,Gabriella,5:30 PM,2017-11-22,Wednesday,1,4,36,16,...,155,177.0,572,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2G,Power,Bri,7:00 AM,2017-11-25,Saturday,1,13,36,10,...,150,173.0,560,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,2G,ESP,Brittney,6:45 PM,2021-07-12,Monday,1,10,26,16,...,143,178.0,500,1.75,3.7,6.4,16:00:00,405.4,NaN,reposting content courtesy of u teapotgohome c...
257,2G,Power,Shania,6:45 PM,2021-07-15,Thursday,2,4,25,24,...,147,175.0,522,1.76,3.7,5,16:04:00,408.3,"The ""Chipper""",reposting content courtesy of u acanthisittafl...
258,3G,Power,Katie,8:15 AM,2021-07-17,Saturday,4,6,24,23,...,144,174.0,508,1.37,4.2,7,14:14:00,71.3,NaN,reposting content courtesy of u rag_monkey cli...
259,2G,Power,Shania,6:45 PM,2021-07-22,Thursday,1,6,25,15,...,148,185.0,524,1.77,4.4,6.4,13:40:00,99.6,12 Minute Prep,reposting content courtesy of u swiftiebean cl...


### Pull a random workout form those pulled, to use on your own
Thought it might be kinda fun to be able to pull a random workout from those that were scraped for use on my own.

In [58]:
wo_list = list(clean_workout.body)

In [61]:
pprint(secrets.choice(wo_list))

('reposting content courtesy of u dc031114 click here to view the comments on '
 'the original post https www reddit com r orangetheory comments l9la78 '
 'monday_1_february_2021_endurance_3g_60_minutes monday 1 february 2021 '
 'endurance 3g 60 minutes respectable template today not much recovery on the '
 'base and the row block will also get your heart rate going not a huge fan of '
 'the floor with the palm to elbows but it wasn t too bad but it won t get '
 'your heart rate up tread block 1 90 sec push 1 min base 2 min push 1 min '
 'base 1 min ao tread block 2 90 sec push 45 sec base 2 min push 45 sec base 1 '
 'min ao row block 14 minutes 600m row 10 x medicine ball ground to press 10 x '
 'iso squat front press 4 x 150m row 10 x medicine ball ground to press 4 x '
 '150m row 10 x medicine ball iso squat from press bonus 150m row 10 x '
 'medicine ball ground to press 10 x iso squat front press floor block 1 6 5 '
 'minutes 5 l x static lunge 5 l x iso lunge to hammer curl 5 r x